<a href="https://colab.research.google.com/github/himalayahall/LargeLanguageModels/blob/main/Tools_and_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/langchain/handbook/00-langchain-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/langchain/handbook/00-langchain-intro.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

The core idea of the library is that we can _"chain"_ together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

* **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

* **LLMs**: Large language models like GPT-3, BLOOM, etc

* **Agents**: Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

* **Memory**: Short-term memory, long-term memory.

# Tools and Chains

This notebook is inspired by the material found on [here](https://www.youtube.com/watch?v=hI2BY7yl_Ac).



In [ ]:
# Install Langchain and OpenAI

!pip install langchain > /dev/null
!pip install openai > /dev/null

### Setup API KEY

In order to access any LLM endpoint - whether it is Open Source or paid - we need the corresponding API Key.

We can use *getpass* to prompt user for the API Key.

In [ ]:
from getpass import getpass

API_KEY = getpass()

··········


In [ ]:
import os

# Store API KEY in environment
os.environ["OPENAI_API_KEY"] = API_KEY

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", max_tokens=256, temperature=0.0)

# Alternatively, open-source LLM hosted on Hugging Face
# pip install huggingface_hub
# from langchain import HuggingFaceHub
# llm = HuggingFaceHub(repo_id = "google/flan-t5-xl", model_kwargs={"temperature":1e-10, "max_length":64})

## Basic LLMChain

### Fact extraction

In [ ]:
article = '''
Coinbase, the second largets crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday showing it's earnings were down 65% to $515 million and operating expenses were up 5% to $750 million.
'''
len(article)

209

In [ ]:
from langchain import PromptTemplate

fact_extractor_template = """
Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Extract the key facts from this text. Don't include opinions. Give each fact a number and keep then short sentences.
Question: {text_input}
Answer: 

"""
fact_extractor_prompt = PromptTemplate(
          template=fact_extractor_template,
          input_variables=['text_input'],
          validate_template=True)

#### Setup Query

In [ ]:
print(
    fact_extractor_prompt.format(
        text_input=article
    )
)


Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Extract the key facts from this text. Don't include opinions. Give each fact a number and keep then short sentences.
Question: 
Coinbase, the second largets crypto exchange by trading volumd, released its Q4 2022 earnings on Tuesday showing it's earnings were down 65% to $515 million and operating expenses were up 5% to $750 million.

Answer: 




#### Create LLMChain

In [ ]:
from langchain import LLMChain

fact_extractor_chain = LLMChain(prompt=fact_extractor_prompt, llm=llm)

#### Call LLM endpoint

In [ ]:
facts = fact_extractor_chain.run(article)

In [ ]:
print(facts)
len(facts)

1. Coinbase is the second largest crypto exchange by trading volume.
2. Coinbase released its Q4 2022 earnings on Tuesday.
3. Coinbase's earnings were down 65% to $515 million.
4. Coinbase's operating expenses were up 5% to $750 million.


237

#### Summarization

In [ ]:
summarization_template = """
Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: You are a financial analyst. Take the following list of facts and write a short summary paragraph for investors. Do not include opinions and do not omit key facts.
Facts: {facts}
Answer: 

"""
summarization_prompt = PromptTemplate(
          template=summarization_template,
          input_variables=['facts'],
          validate_template=True)

In [ ]:
investor_update_chain = LLMChain(prompt=summarization_prompt, llm=llm)

investor_update = investor_update_chain.run(facts)

print(investor_update)
len(investor_update)

Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.


206

### Sequential chains

In [ ]:
from langchain.chains import SequentialChain, SimpleSequentialChain

In [ ]:
full_chain = SimpleSequentialChain(chains=[fact_extractor_chain, investor_update_chain], verbose=True)
response = full_chain.run(article)



> Entering new SimpleSequentialChain chain...
1. Coinbase is the second largest crypto exchange by trading volume.
2. Coinbase released its Q4 2022 earnings on Tuesday.
3. Coinbase's earnings were down 65% to $515 million.
4. Coinbase's operating expenses were up 5% to $750 million.
Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.

> Finished chain.


In [ ]:
print(response)
len(response)

Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.


206

### Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

conv_buffer_memory = ConversationBufferMemory()

In [ ]:
full_chain = SimpleSequentialChain(chains=[fact_extractor_chain, investor_update_chain], memory=conv_buffer_memory, verbose=True)
response = full_chain.run(article)



> Entering new SimpleSequentialChain chain...
1. Coinbase is the second largest crypto exchange by trading volume.
2. Coinbase released its Q4 2022 earnings on Tuesday.
3. Coinbase's earnings were down 65% to $515 million.
4. Coinbase's operating expenses were up 5% to $750 million.
Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.

> Finished chain.


In [ ]:
print(response)
print(len(response))
print(conv_buffer_memory)

Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.
206
chat_memory=ChatMessageHistory(messages=[HumanMessage(content="\nCoinbase, the second largets crypto exchange by trading volumd, released its Q4 2022 earnings on Tuesday showing it's earnings were down 65% to $515 million and operating expenses were up 5% to $750 million.\n", additional_kwargs={}, example=False), AIMessage(content='Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'


In [ ]:
for m in conv_buffer_memory.chat_memory.messages:
  print(f'{m.type}: {m.content}')


human: 
Coinbase, the second largets crypto exchange by trading volumd, released its Q4 2022 earnings on Tuesday showing it's earnings were down 65% to $515 million and operating expenses were up 5% to $750 million.

ai: Coinbase, the second largest crypto exchange by trading volume, released its Q4 2022 earnings on Tuesday. The earnings were down 65% to $515 million, while the operating expenses were up 5% to $750 million.


In [ ]:
from typing import TypeVar, Generic

T = TypeVar('T')

class Stack(Generic[T]):
    def __init__(self) -> None:
        # Create an empty list with items of type T
        self.items: list[T] = []

    def push(self, item: T) -> None:
        self.items.append(item)

    def pop(self) -> T:
        return self.items.pop()

    def empty(self) -> bool:
        return not self.items

In [ ]:
# Construct an empty Stack[int] instance
stack = Stack[int]()
stack.push(2)
stack.pop()
stack.push('x') 